In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [289]:
dt = pd.read_csv('all-data.csv')
dt.head()

,num,sentiment,review
0,1,neutral,"According to Gran , the company has no plans t..."
1,2,neutral,Technopolis plans to develop in stages an area...
2,3,negative,The international electronic industry company ...
3,4,positive,With the new production plant the company woul...
4,5,positive,According to the company 's updated strategy f...


Reemplazamos los negativos por el numero 1, los neutrales por el numero 2 y los positivos por el numero 3 esto para poder sacar la polaridad de los datos.

In [290]:
# dt = dt.replace("positive", 1)
# dt = dt.replace("neutral", 2)
# dt = dt.replace("negative", 3)
# dt.head()

In [291]:
dt_clean = dt

dt_clean.drop(columns = 'num', inplace = True)

# Limpieza de datos

stop = stopwords.words('english')

#Quitamos puntos, porcentajes, comas, etc etc y volvemos todo el texto a minusculas.
dt_clean['review'] = dt_clean['review'].str.replace("[^\w\s]", "").str.lower()

#Quitamos todos los valores numericos
dt_clean['review'] = dt_clean['review'].str.replace("[^\D]", "").str.lower()

#Quitamos las stopwords
#dt_clean['review'] = dt_clean['review'].apply(lambda x: [item for item in x.split() if item not in stop])

print(dt_clean)

     sentiment                                             review
0      neutral  according to gran  the company has no plans to...
1      neutral  technopolis plans to develop in stages an area...
2     negative  the international electronic industry company ...
3     positive  with the new production plant the company woul...
4     positive  according to the company s updated strategy fo...
...        ...                                                ...
4841  negative  london marketwatch  share prices ended lower i...
4842   neutral  rinkuskiai s beer sales fell by  per cent to  ...
4843  negative  operating profit fell to eur  mn from eur  mn ...
4844  negative  net sales of the paper segment decreased to eu...
4845  negative  sales in finland decreased by   in january  wh...

[4846 rows x 2 columns]


In [292]:
dt_clean.sentiment.value_counts()

neutral     2879
positive    1363
negative     604
Name: sentiment, dtype: int64

In [293]:
sent = SentimentIntensityAnalyzer()

dt_clean['scores'] = dt_clean['review'].apply(lambda review: sent.polarity_scores(review))

dt_clean.head(5)

,sentiment,review,scores
0,neutral,according to gran the company has no plans to...,"{'neg': 0.092, 'neu': 0.837, 'pos': 0.071, 'co..."
1,neutral,technopolis plans to develop in stages an area...,"{'neg': 0.075, 'neu': 0.925, 'pos': 0.0, 'comp..."
2,negative,the international electronic industry company ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,positive,with the new production plant the company woul...,"{'neg': 0.038, 'neu': 0.66, 'pos': 0.302, 'com..."
4,positive,according to the company s updated strategy fo...,"{'neg': 0.0, 'neu': 0.825, 'pos': 0.175, 'comp..."


In [294]:
dt_clean['score_neg']  = dt_clean['scores'].apply(lambda score_dict: score_dict['neg'])
dt_clean['score_neu']  = dt_clean['scores'].apply(lambda score_dict: score_dict['neu'])
dt_clean['score_pos']  = dt_clean['scores'].apply(lambda score_dict: score_dict['pos'])
dt_clean['compound']  = dt_clean['scores'].apply(lambda score_dict: score_dict['compound'])
dt_clean.drop(columns = 'scores', inplace = True)

dt_clean.head()

,sentiment,review,score_neg,score_neu,score_pos,compound
0,neutral,according to gran the company has no plans to...,0.092,0.837,0.071,-0.1280
1,neutral,technopolis plans to develop in stages an area...,0.075,0.925,0.000,-0.2960
2,negative,the international electronic industry company ...,0.000,1.000,0.000,0.0000
3,positive,with the new production plant the company woul...,0.038,0.660,0.302,0.8555
4,positive,according to the company s updated strategy fo...,0.000,0.825,0.175,0.6705


# NO LE HAGAN CASO A ESTE

In [223]:
dt_clean['true_sentiment'] = dt_clean['compound'].apply(lambda c: 'positive' if c >= 0.7 else ('neutral' if c >= 0 and c <= 0.69 else 'negative'))

dt_clean.head(5)

,sentiment,review,scores,compound,true_sentiment
0,neutral,according to gran the company has no plans to...,"{'neg': 0.092, 'neu': 0.837, 'pos': 0.071, 'co...",-0.1280,negative
1,neutral,technopolis plans to develop in stages an area...,"{'neg': 0.075, 'neu': 0.925, 'pos': 0.0, 'comp...",-0.2960,negative
2,negative,the international electronic industry company ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
3,positive,with the new production plant the company woul...,"{'neg': 0.038, 'neu': 0.66, 'pos': 0.302, 'com...",0.8555,positive
4,positive,according to the company s updated strategy fo...,"{'neg': 0.0, 'neu': 0.825, 'pos': 0.175, 'comp...",0.6705,neutral
5,positive,financing of aspocomp s growth aspocomp is agg...,"{'neg': 0.157, 'neu': 0.662, 'pos': 0.181, 'co...",0.1796,neutral
6,positive,for the last quarter of componenta s net sal...,"{'neg': 0.067, 'neu': 0.848, 'pos': 0.085, 'co...",0.1531,neutral
7,positive,in the third quarter of net sales increased ...,"{'neg': 0.0, 'neu': 0.773, 'pos': 0.227, 'comp...",0.6124,neutral
8,positive,operating profit rose to eur mn from eur mn ...,"{'neg': 0.0, 'neu': 0.854, 'pos': 0.146, 'comp...",0.4404,neutral
9,positive,operating profit totalled eur mn up from eur...,"{'neg': 0.0, 'neu': 0.818, 'pos': 0.182, 'comp...",0.4404,neutral


In [260]:
dt_clean['true_sentiment'] = dt_clean.apply(lambda x: 'positive' if x['score_pos'] > x['score_neg'] and x['score_pos'] > x['score_neu'] else ('negative' if x['score_neg'] > x['score_pos'] and x['score_neg'] > x['score_neu'] else 'neutral'), axis = 1)

In [261]:
dt_clean.head(5)

,sentiment,review,scores,compound,score_neg,score_neu,score_pos,true_sentiment
0,neutral,according to gran the company has no plans to...,"{'neg': 0.092, 'neu': 0.837, 'pos': 0.071, 'co...",-0.1280,0.092,0.837,0.071,neutral
1,neutral,technopolis plans to develop in stages an area...,"{'neg': 0.075, 'neu': 0.925, 'pos': 0.0, 'comp...",-0.2960,0.075,0.925,0.000,neutral
2,negative,the international electronic industry company ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,0.000,1.000,0.000,neutral
3,positive,with the new production plant the company woul...,"{'neg': 0.038, 'neu': 0.66, 'pos': 0.302, 'com...",0.8555,0.038,0.660,0.302,neutral
4,positive,according to the company s updated strategy fo...,"{'neg': 0.0, 'neu': 0.825, 'pos': 0.175, 'comp...",0.6705,0.000,0.825,0.175,neutral


In [262]:
dt_clean.true_sentiment.value_counts()

neutral     4825
positive      19
negative       2
Name: true_sentiment, dtype: int64